In [1]:
import json
import re
from gensim.models import Word2Vec
import csv
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
#import snowballstemmer

In [3]:
f = open('digital_trajectoriesraject_public_keyword.json')
digital_trajectoriesraject_public_keyword = json.load(f)

f.close()

In [4]:
f = open('documents1.json')
documents = json.load(f)

f.close()

In [5]:
duplicates_data = pd.read_csv('duplicates_data.csv')
duplicates_data

,0,1,2
0,нереляционными субд,nosql базы данных,1
1,систем управления базами данных,системах управления базами данных,1
2,примере субд,система управления базами данных,1
3,разработке баз данных,реализации бд,1
4,субд,система управления базами данных,1
5,объектно-ориентированным программированием,ооп,1
6,бд,баз данных,1
7,big data,больших данных,1
8,использованием ros,применение пакетов ros,1
9,интеллектуального анализа данных,data mining,1


https://python-school.ru/blog/word2vec-with-examples-in-gensim/

## digital_trajectoriesraject_public_keyword

In [6]:
text = ""
for doc in documents: text += doc['text']

In [7]:
sentences = re.split(r'\.+\s', text.lower())


In [8]:
sentences[:3]

['коммерциализация результатов научных исследований и разработок представляет собой процесс вовлечения их в экономический (коммерческий) оборот в различных сегментах национального и глобального рынков',
 'актуальность данной дисциплины обусловлена становлением на путь модернизации экономики страны',
 'в сложившейся ситуации резко возрастает необходимость оценки и обоснования экономической эффективности проектов, ориентированных на выпуск высокотехнологичной продукции и продвижение новых технологий']

In [9]:
sentences_word_by_word = []
for i in sentences:
    i = re.sub(r'[^а-яёa-z -+.\/0-9]', '', i)
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    sentences_word_by_word.append(i.split(' '))

In [10]:
arr = []
for i in digital_trajectoriesraject_public_keyword:
    i = i['value']
    i = re.sub('[^а-яёa-z -+.\/0-9]', '', i.lower())
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    arr.append(i)
    
digital_trajectoriesraject_public_keyword = list(set([] + arr))

len(digital_trajectoriesraject_public_keyword), digital_trajectoriesraject_public_keyword[:5]

(21727,
 ['электрохимические процессы',
  'продукт',
  'computational geometry',
  'морской среде',
  'изучение законов управления'])

In [11]:
w2v_model = Word2Vec(
    min_count=1,
    window=5,
    negative=0,
    alpha=0.1,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=-1)

#hs поменял вывод

In [12]:
w2v_model.build_vocab(sentences_word_by_word)

In [13]:
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(81898012, 114943620)

In [14]:
w2v_model.init_sims(replace=True)


/tmp/ipykernel_11364/2897808894.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [15]:
w2v_model.save('w2v_model.model')

In [16]:
#w2v_model = Word2Vec.load('w2v_model.model')

In [17]:
X_test = []
dictionary_test = {}
notInVocab_test = []

for s in duplicates_data.iloc[:,:2].values:
    for phrase in s:
        vec = 0
        flag = 0
        for word in phrase.split(' '):
            if not word in w2v_model.wv.key_to_index.keys():
                flag = 1
                notInVocab_test.append(word)
                break
            vec += w2v_model.wv[word]
        vec /= len(phrase.split(' '))
        if flag:
            continue
        string = " ".join(str(v) for v in vec)
        if string in dictionary_test.keys():
            dictionary_test[string].append(phrase)
        else:
            dictionary_test[string] = [phrase]
        X_test.append(vec)
    
#X_test[0]

In [18]:
import numpy as np
data_len = len(X_test) // 2
arr1 = []
arr0 = []
for i in range(data_len):
    point_1 = X_test[i * 2]
    point_2 = X_test[i * 2 + 1]
    distance = np.sqrt(sum((a-b) ** 2 for a, b in zip(point_1, point_2)))
    if duplicates_data.iloc[i, 2] == 1:
        arr1.append(distance)
    else:
        arr0.append(distance)


In [19]:
min1 = round(min(arr1), 4)
max1 = round(max(arr1), 4)
mean1 = round(np.array(arr1).mean(), 4)
med1 = round(np.median(np.array(arr1)), 4)

print('min =', min1, 
      'max =', max1, 
      'mean =', mean1,
      'median =', med1)

arr1, sorted(arr1)


min = 0.199 max = 1.2751 mean = 0.9028 median = 0.926


([0.8438082893917027,
  0.19904198359480776,
  0.8057130306449952,
  0.7724049150053682,
  0.9468181648436754,
  1.2750824412119388,
  1.019891694648357,
  0.9513641037726156,
  0.8396055671201867,
  1.0660934606162717,
  1.2477795949441173,
  0.7562163417745683,
  1.0101850505060563,
  0.9051086934418493],
 [0.19904198359480776,
  0.7562163417745683,
  0.7724049150053682,
  0.8057130306449952,
  0.8396055671201867,
  0.8438082893917027,
  0.9051086934418493,
  0.9468181648436754,
  0.9513641037726156,
  1.0101850505060563,
  1.019891694648357,
  1.0660934606162717,
  1.2477795949441173,
  1.2750824412119388])

In [20]:
min0 = round(min(arr0), 4)
max0 = round(max(arr0), 4)
mean0 = round(np.array(arr0).mean(), 4)
med0 = round(np.median(np.array(arr0)), 4)

print('min =', min0, 
      'max =', max0, 
      'mean =', mean0,
      'median =', med0)

arr0, sorted(arr0)

min = 0.6943 max = 1.2323 mean = 0.9346 median = 0.937


([1.015142904920445,
  0.9371668011039482,
  0.7106318968966491,
  0.6942758968549447,
  1.0689782503877674,
  0.8665332392494606,
  0.8019737076297703,
  0.8505863576605099,
  1.005206419449717,
  0.9808398063038244,
  1.1124875315733065,
  0.9192100486822079,
  0.9370308306344446,
  1.2322526159283982,
  0.8862166241393746],
 [0.6942758968549447,
  0.7106318968966491,
  0.8019737076297703,
  0.8505863576605099,
  0.8665332392494606,
  0.8862166241393746,
  0.9192100486822079,
  0.9370308306344446,
  0.9371668011039482,
  0.9808398063038244,
  1.005206419449717,
  1.015142904920445,
  1.0689782503877674,
  1.1124875315733065,
  1.2322526159283982])

In [21]:
print('min0-min1=', round(min0 - min1, 4), 
      '\nmax0-max1=', round(max0 - max1, 4), 
      '\nmean0-mean1=', round(mean0 - mean1, 4), 
      '\nmedian0-median1=', round(med0 - med1, 4))

min0-min1= 0.4953 
max0-max1= -0.0428 
mean0-mean1= 0.0318 
median0-median1= 0.011


In [22]:
X = []
dictionary = {}
notInVocab = []

for phrase in digital_trajectoriesraject_public_keyword:
    vec = 0
    flag = 0
    #print(phrase)
    for word in phrase.split(' '):
        if not word in w2v_model.wv.key_to_index.keys():
            flag = 1
            notInVocab.append(word)
            break
        vec += w2v_model.wv[word]
    vec /= len(phrase.split(' '))
    if flag:
        continue
    string = " ".join(str(v) for v in vec)
    if string in dictionary.keys():
        dictionary[string].append(phrase)
    else:
        dictionary[string] = [phrase]
        X.append(vec)
    
    
#X[0]

### DBSCAN

In [23]:
clustering = DBSCAN(eps=0.05, min_samples=2, metric='cosine').fit(X)


In [24]:
clustering.labels_

array([-1, -1, -1, ..., -1, -1, -1])

In [25]:
number_of_clusters = clustering.labels_.max()

for i in range(-1, number_of_clusters):
    print(list(clustering.labels_).count(i))

20763
2
51
2
4
2
3
6
12
2
2
16
28
9
2
6
2
2
15
2
10
2
2
5
2
3
2
3
2
2
2
2
2
2
3
2
2
8
8
2
3
2
7
7
2
2
2
3
2
2
2
16
3
2
3
2
2
2
2
2
2
3
3
2
2
2
2
2
2
3
3
2
2
3
2
2
2
2
2
2
3
2
9
3
2
2
2
2
2
2
3
2
2
2
2
4
2
2
2
3
2
3
2
2
2
3
3
2
2
2
2
2
2
2
3
3
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
4
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
2
3
4
2
3
2
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
3
2
2
2
4
3
2
3
2
2
2
2
2
4
2
2
2
5
3
2
2
2
2
2
3
2
2
3
2
2
2
2
2
3
2
3
2
2
2
4
2
2
2
2
2
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2


In [26]:
synonyms_array = []
size = len(X)
labels = list(clustering.labels_)

for i in range(number_of_clusters):
    cluster = np.where(clustering.labels_ == i)[0]
    synonyms = []
    for vec in cluster:
        synonyms.extend(dictionary[" ".join([str(v) for v in X[vec]])])
    if len(synonyms) != len(list(set(synonyms))):
        print('cluster:', cluster)
        print(synonyms)
    synonyms_array.append(synonyms)
            

In [30]:
synonyms_json = []
for i in range(len(synonyms_array)):
    synonyms_json.append({'id': i,
                          'cluster': synonyms_array[i]})
    print(i, ' (', len(synonyms_array[i]), ')', sep='')
    for j in synonyms_array[i][:20]:
        print(j)
    if len(synonyms_array[i]) > 20:
        print('...')
    print()

0 (2)
тока
постоянного тока

1 (52)
освоение навыков методов
формирование умений использования принципов
навыками применения методов
методов применения
применения методов
формирование умений применения
формирование умений применения методов решения
формирование умений использования методов
освоение навыков использования моделей
использования методов
навыков использования методов
освоение методов
освоение навыков применения методов
освоение навыков выбора методов
навыков применения методов анализа
навыками применения методов проектирования
приобретение навыков использования моделей
навыков применения
применения методов построения
получение навыков применения методов
...

2 (2)
навыков выполнения схем
формирование навыков выполнения схем

3 (4)
результате освоения
результате освоения дисциплины
результате освоения дисциплины магистрант
результате освоения дисциплины учащиеся

4 (2)
infinity alarm
infinity trend

5 (3)
методов оптимизации систем управления
оптимизации систем управления эл

In [32]:
parameters = {'eps': 0.05,
             'min_count': 1,
             'window': 5,
             'negative': 0,
             'alpha': 0.1,
             'min_alpha': 0.005,
             'sample': 1e-4,
             'sg': 1,
             'hs': 1
             }
import json

with open('synonyms.json', 'w') as outfile:
    outfile.write(json.dumps({'params': parameters, 'synonyms': synonyms_json}, ensure_ascii=False, indent=4))

Зачастую в один кластер попадают подстроки

In [28]:
for i in list(dictionary.keys()):
    if len(dictionary[i]) > 1:
        print(dictionary[i], '\n')

['излучения рентгеновского', 'рентгеновского излучения'] 

['проектирования объектов', 'объектов проектирования'] 

['классификации систем', 'систем классификации'] 

['методов применения', 'применения методов'] 

['свч фильтры', 'фильтры свч'] 

['устройствах свч', 'свч устройствах'] 

['методов организации', 'организации методов'] 

['систем автономных', 'автономных систем'] 

['алгоритмов оптимизации', 'оптимизации алгоритмов'] 

['моделирования систем', 'систем моделирования'] 

['самостоятельного проведения', 'проведения самостоятельного'] 

['систем информационных', 'информационных систем'] 

['представления моделей', 'моделей представления'] 

['tcad synopsys', 'synopsys tcad'] 

['работы навыков', 'навыков работы'] 

['систем проектирования', 'проектирования систем'] 

['дискретных случайных сигналов', 'случайных дискретных сигналов'] 

['устройств свч', 'свч устройств'] 

['средств проектирования', 'проектирования средств'] 

['управления процессами', 'процессами управления'] 

## При вычислении среднего из векторов

### 1


## При суммировании векторов

### Вариант n
при 

    min_count=1,
    window=5,
    negative=0,
    alpha=0.7,
    min_alpha=0.01,
    sample=1e-1,
    sg=1,
    hs=1
    
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

для 1: min = 0.2162 max = 3.7256 mean = 1.9104 median = 1.9625

для 0: min = 0.9745 max = 3.6274 mean = 2.0252 median = 2.1101

**0.2162** бд - баз данных

**3.7256** нереляционными субд	nosql базы данных	
 
 
**0.9745** оценку структуры	другие методы

**3.6274** систем моделирования	методы исследования свойств клеток


### Вариант №m

min0-min1= 0.3275

max0-max1= 0.6789 

mean0-mean1= 0.3233 

median0-median1= 0.1946

w2v_model = Word2Vec(

    min_count=1,
    window=5,
    negative=0,
    alpha=0.5,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=1)
    
w2v_model.train(sentences_word_by_word,total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

1: min = 0.6212 max = 2.8116 mean = 1.7783 median = 1.7289

0: min = 0.9487 max = 3.4905 mean = 2.1016 median = 1.9235